# Modeling with StatsModels

## 1. Ordinary Least Square
- Column Names
- Log Transformation
- Condition Number
- Standard Scaling

# 2. Dimensionality Reduction
- ANOVA
- F-test and Feature Influence

# 3. Outlier
- Cook's Distance

# 4. Regularization
- Lasso

# 5. Diagnosis of Regression
- Residual Normality Test
- Partial Regression Plot

# 6. Cross Validatoin

# 7. Test
- score

In [ ]:
# %matplotlib inline
# pd.options.mode.chained_assignment = None
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 500)
# import warnings
# import sys
# import datetime
# import scipy as sp
# import statsmodels.stats.api as sms
# import statsmodels.api as sm
# from patsy import dmatrix
# from sklearn.base import BaseEstimator, RegressorMixin
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from sklearn.metrics import r2_score
# import utils.statsmodel_helper as sh
# import utils.feature_selection as fs
# import utils.preprocessing as pp
# import utils.error_calculator as ec


# class SMWrapper(BaseEstimator, RegressorMixin):
#     """ A universal sklearn-style wrapper for statsmodels regressors """
#     def __init__(self, model_class, fit_intercept=True):
#         self.model_class = model_class
#         self.fit_intercept = fit_intercept
#     def fit(self, X, y):
#         if self.fit_intercept:
#             X = sm.add_constant(X)
#         self.model_ = self.model_class(y, X)
#         self.results_ = self.model_.fit()
#     def predict(self, X):
#         if self.fit_intercept:
#             X = sm.add_constant(X)
#         return self.results_.predict(X)



# sys.setrecursionlimit(1500)

# degree = 2
# skewness_limit = 1
# num_of_cooks = 2
# num_of_f_test = 10

In [1]:
import warnings
warnings.simplefilter("ignore")
import statsmodels.api as sm

# modeling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Linear Regression Model - sklearn
from sklearn.model_selection import train_test_split

In [2]:



df_train_macro = pd.read_csv('../code/data/train_macro3.csv', index_col=0)
df_test_macro = pd.read_csv('../code/data/test_macro3.csv', index_col=0)

In [3]:
# numeric_ivs = df_train_macro._get_numeric_data().columns.drop('price_doc').tolist()
numeric_ivs = df_train_macro._get_numeric_data().columns.tolist()
categorial_ivs = list(df_train_macro.dtypes[df_train_macro.dtypes == object].index)

In [4]:
df_cat_dummies = pd.get_dummies(df_train_macro[categorial_ivs])

In [5]:
df_real = df_train_macro[numeric_ivs]

In [6]:
df_train_macro = pd.concat([df_real, df_cat_dummies], axis=1)
cols = list(df_train_macro.columns.values)
cols.pop(cols.index('price_doc'))
df_train_macro = df_train_macro[cols + ['price_doc']]
df_train_macro.tail(2)

,usdrub,full_sq,life_sq,floor,num_room,kitch_sq,state,area_m,preschool_education_centers_raion,school_education_centers_raion,...,big_road1_1line_no,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory,price_doc
id,,,,,,,,,,,,,,,,,,,,,
30472,55.2655,64,32.0,5.0,2.0,11.0,2.0,6050064.566,6,8,...,1,0,1,0,0,0,0,0,1,13500000
30473,55.2655,43,28.0,1.0,2.0,6.0,2.0,4395332.782,4,4,...,1,0,1,0,0,0,0,1,0,5600000


In [7]:
dfy = df_train_macro.iloc[:, -1]
dfx = df_train_macro.iloc[:, :-1]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.3, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(21282, 9122, 21282, 9122)

In [9]:
# StandardScaler
std_scaler = StandardScaler()
std_scaler.fit(X_train)
X_train_std_scaler = std_scaler.transform(X_train)
std_scaler.fit(X_test)
X_test_std_scaler = std_scaler.transform(X_test)

In [10]:
# RobustScaler
robust_scaler = RobustScaler()
robust_scaler.fit(X_train)
X_train_robust_scaler = robust_scaler.transform(X_train)
robust_scaler.fit(X_test)
X_train_robust_scaler = robust_scaler.transform(X_test)

In [11]:
X_train = sm.add_constant(X_train)

In [12]:
result = sm.OLS(y_train, X_train).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              price_doc   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.592
Method:                 Least Squares   F-statistic:                     195.3
Date:                Fri, 15 Nov 2019   Prob (F-statistic):               0.00
Time:                        17:57:50   Log-Likelihood:            -3.4767e+05
No. Observations:               21282   AIC:                         6.957e+05
Df Residuals:                   21122   BIC:                         6.969e+05
Df Model:                         159                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                 -7.516e+05   1.75e+05     -4.297      0.000   -1.09e+06   -4.09e+05
usdrub                                 4.691e+04   2242.703     20.917      0.000    4.25e+04    5.13e+04
full_sq                                 1.22e+05   2074.826     58.806      0.000    1.18e+05    1.26e+05
life_sq                                2.132e+04   2592.657      8.222      0.000    1.62e+04    2.64e+04
floor                                  5.386e+04   4289.611     12.556      0.000    4.55e+04    6.23e+04
num_room                              -5.136e+05   6.18e+04     -8.316      0.000   -6.35e+05   -3.93e+05
kitch_sq                               2.841e+04   5780.290      4.915      0.000    1.71e+04    3.97e+04
state                                  4.936e+05   4.23e+04     11.669      0.000    4.11e+05    5.77e+05
area_m                                   -0.0112      0.004     -3.026      0.002      -0.019      -0.004
preschool_education_centers_raion      1.085e+05   4.56e+04      2.382      0.017    1.92e+04    1.98e+05
school_education_centers_raion        -8.251e+04   3.55e+04     -2.325      0.020   -1.52e+05   -1.29e+04
school_education_centers_top_20_raion -2016.8970   9.75e+04     -0.021      0.983   -1.93e+05    1.89e+05
hospital_beds_raion                      95.6188     40.195      2.379      0.017      16.833     174.404
healthcare_centers_raion               -1.88e+04   2.78e+04     -0.676      0.499   -7.33e+04    3.57e+04
university_top_20_raion               -1.059e+05   1.04e+05     -1.019      0.308    -3.1e+05    9.79e+04
sport_objects_raion                   -3.197e+04   1.24e+04     -2.576      0.010   -5.63e+04   -7639.853
shopping_centers_raion                 1.441e+04   1.46e+04      0.988      0.323   -1.42e+04     4.3e+04
office_raion                           5132.8279   6238.794      0.823      0.411   -7095.684    1.74e+04
0_17_all                                 -4.5920      8.795     -0.522      0.602     -21.830      12.646
raion_build_count_with_material_info   2.135e+04    2.9e+04      0.736      0.462   -3.55e+04    7.82e+04
build_count_brick                       850.8363    729.461      1.166      0.243    -578.962    2280.635
build_count_monolith                  -2288.7150   3246.679     -0.705      0.481   -8652.455    4075.025
raion_build_count_with_builddate_info -2.092e+04   2.91e+04     -0.719      0.472   -7.79e+04    3.61e+04
build_count_before_1920                -156.8456   1285.948     -0.122      0.903   -2677.402    2363.711
metro_min_avto                        -5.095e+04   1.86e+04     -2.737      0.006   -8.74e+04   -1.45e+04
kindergarten_km                       -7.367e+04   4.49e+04     -1.642      0.101   -1.62e+05    1.43e+04
school_km

In [13]:
X_train_std_scaler = sm.add_constant(X_train_std_scaler)
result_std_scaler = sm.OLS(y_train, X_train_std_scaler).fit()
result_std_scaler.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              price_doc   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.592
Method:                 Least Squares   F-statistic:                     195.3
Date:                Fri, 15 Nov 2019   Prob (F-statistic):               0.00
Time:                        17:57:51   Log-Likelihood:            -3.4767e+05
No. Observations:               21282   AIC:                         6.957e+05
Df Residuals:                   21122   BIC:                         6.969e+05
Df Model:                         159                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       7.135e+06   2.07e+04    344.574      0.000    7.09e+06    7.18e+06
x1          4.495e+05   2.15e+04     20.917      0.000    4.07e+05    4.92e+05
x2          2.717e+06   4.62e+04     58.806      0.000    2.63e+06    2.81e+06
x3          4.202e+05   5.11e+04      8.222      0.000     3.2e+05     5.2e+05
x4          2.852e+05   2.27e+04     12.556      0.000    2.41e+05     3.3e+05
x5          -4.04e+05   4.86e+04     -8.316      0.000   -4.99e+05   -3.09e+05
x6          1.207e+05   2.45e+04      4.915      0.000    7.25e+04    1.69e+05
x7          3.328e+05   2.85e+04     11.669      0.000    2.77e+05    3.89e+05
x8         -2.357e+05   7.79e+04     -3.026      0.002   -3.88e+05    -8.3e+04
x9          3.253e+05   1.37e+05      2.382      0.017    5.76e+04    5.93e+05
x10         -2.85e+05   1.23e+05     -2.325      0.020   -5.25e+05   -4.47e+04
x11         -669.7723   3.24e+04     -0.021      0.983   -6.41e+04    6.28e+04
x12         7.786e+04   3.27e+04      2.379      0.017    1.37e+04    1.42e+05
x13        -2.806e+04   4.15e+04     -0.676      0.499   -1.09e+05    5.33e+04
x14        -4.674e+04   4.59e+04     -1.019      0.308   -1.37e+05    4.32e+04
x15        -2.104e+05   8.17e+04     -2.576      0.010    -3.7e+05   -5.03e+04
x16         6.818e+04    6.9e+04      0.988      0.323   -6.71e+04    2.03e+05
x17         1.203e+05   1.46e+05      0.823      0.411   -1.66e+05    4.07e+05
x18        -4.263e+04   8.16e+04     -0.522      0.602   -2.03e+05    1.17e+05
x19         5.581e+06   7.59e+06      0.736      0.462   -9.29e+06    2.05e+07
x20          1.01e+05   8.66e+04      1.166      0.243   -6.87e+04    2.71e+05
x21        -4.048e+04   5.74e+04     -0.705      0.481   -1.53e+05    7.21e+04
x22        -5.469e+06    7.6e+06     -0.719      0.472   -2.04e+07    9.43e+06
x23        -8847.6346   7.25e+04     -0.122      0.903   -1.51e+05    1.33e+05
x24        -3.357e+05   1.23e+05     -2.737      0.006   -5.76e+05   -9.53e+04
x25        -1.634e+05   9.95e+04     -1.642      0.101   -3.58e+05    3.17e+04
x26         8.377e+05   7.72e+05      1.086      0.278   -6.75e+05    2.35e+06
x27        -1.779e+05   1.31e+05     -1.353      0.176   -4.36e+05    7.98e+04
x28        -4.473e+05    9.1e+04     -4.915      0.000   -6.26e+05   -2.69e+05
x29         2.922e+05   9.42e+04      3.103      0.002    1.08e+05    4.77e+05
x30         1.396e+05   8.03e+04      1.738      0.082   -1.79e+04    2.97e+05
x31         1.852e+06   9.88e+05      1.876      0.061   -8.33e+04    3.79e+06
x32        -4.471e+06   2.44e+06     -1.834      0.067   -9.25e+06    3.07e+05
x33         5.731e+06    2.2e+06      2.606      0.009    1.42e+06       1e+07
x34        -2.661e+06   2.78e+06     -0.957      0.339   -8.11e+06    2.79e+06
x35        -1.286e+05   4.68e+04     -2.749      0.006    -2.2e+05   -3.69e+04
x3

In [17]:
# X_train_robust_scaler = sm.add_constant(X_train_robust_scaler)
# result_robust_scaler = sm.OLS(y_train, X_train_robust_scaler).fit()
# result_robust_scaler.summary()

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression()),
])

## Column Names

In [ ]:
new_cols = []
for col in  list(df_train_macro.columns):
    col = col.replace('-', '_').replace('+', '_').replace(':', '_').replace('~', '_').replace('*', '_')
    new_cols.append('_'+col)
df_train_macro.columns = new_cols

new_cols = []
for col in list(df_test_macro.columns):
    col = col.replace('-', '_').replace('+', '_').replace(':', '_').replace('~', '_').replace('*', '_')
    new_cols.append('_'+col)
df_test_macro.columns = new_cols

categorial_ivs = list(set(df_train_macro.columns) - set(df_train_macro._get_numeric_data().columns))
numeric_ivs = df_train_macro._get_numeric_data().columns.drop('_price_doc').tolist()

## Log Transformation
Transform data with skewness greater than 1.

In [ ]:
features_to_log = []
for f in df_train_macro._get_numeric_data().columns:
    skewness = sp.stats.skew(df_train_macro[f])
    if skewness > skewness_limit:
        features_to_log.append(f)

for col in df_train_macro._get_numeric_data().columns:
    if col != '_price_doc':
        min_val_train = min(df_train_macro[col])
        min_val_test  = min(df_test_macro[col])
        min_val = min(min_val_train, min_val_test)
        if min_val <= 0:
            df_train_macro[col] += (np.abs(min_val) + 0.1)
            df_test_macro[col]  += (np.abs(min_val) + 0.1)
    else:
        min_val_train = min(df_train_macro[col])
        if min_val_train <= 0:
            df_train_macro[col] += (np.abs(min_val_train) + 0.1)

In [ ]:
formula = sh.make_statsmodels_ols_formula(numeric_ivs, categorial_ivs, '_price_doc', log_vs=features_to_log, degree=degree, scale=False)
model = sm.OLS.from_formula(formula, data=df_train_macro)
result = model.fit()
result.summary()

## Condition Number
Large condition number occurs when the scale of data changes significantly due to the unit difference. Scaling can decrease condition number. Multicollinearity can also cause large condition number. We can handle this by reducing dimensionality with variance inflation factor.

## Standard Scaling
Standalize variables by removing the mean and scaling to unit variance.

In [ ]:
formula = sh.make_statsmodels_ols_formula(numeric_ivs, categorial_ivs, '_price_doc', log_vs=features_to_log, degree=degree, scale=True)
model = sm.OLS.from_formula(formula, data=df_train_macro)
result = model.fit()
result.summary()

Scaling did not significantly decrease the condition number.

# 2. Dimensionality Reduction
## ANOVA

In [ ]:
anova = sm.stats.anova_lm(result, typ=2)
anova


We can remove features with p-value equal or greater than 0.05 since they have very small influences on the dependent variable

## F-test and Feature Influence

In [ ]:
result, sms_vars, formula = fs.by_f_test(df_train_macro, formula, repeat=num_of_f_test)
result.summary()

# 3. Outlier
## Cook's Distance
- Find data with large leverage and residual by calculating Cook's distance.

In [ ]:
df_train_macro_with_outliers = df_train_macro.copy(deep=True)
df_train_macro, model, result = pp.remove_outliers(df_train_macro, formula, repeat=3)
result.summary()

# 4. Regularization
## Lasso
Find variables with zero coefficient when Lasso regularization is applied.

In [ ]:
result_lasso = model.fit_regularized(alpha=0.001, L1_wt=1)

Let's remove features with zero coefficient to reduce dimensionality.

In [ ]:
sms_vars = []
for idx, coef in enumerate(result_lasso.params):
    if coef ==0:
        continue
    feature = result_lasso.params.index[idx]
    if feature == 'Intercept':
        continue
    startDelPos = feature.find('[')
    endDelPos = feature.find(']')
    feature = feature.replace(feature[startDelPos:endDelPos+1], '')
    sms_vars.append(feature)

In [ ]:
formula = 'np.log(_price_doc) ~ ' + " + ".join(sms_vars)
model =sm.OLS.from_formula(formula, data=df_train_macro)
result = model.fit()
result.summary()

# 5. Diagnosis of Regression
## Residual Normality Test

In [ ]:
# outlier remove result 
sp.stats.probplot(result.resid, plot=plt)
plt.show()

In [ ]:
test = sms.omni_normtest(result.resid)
for xi in zip(['Chi^2', 'P-value'], test):
    print("%-12s: %6.3f" % xi)

# Partial Regression Plot
Let's visualize the influence of a single independent variable.

In [ ]:
fig = plt.figure(figsize=(10,70))
sm.graphics.plot_partregress_grid(result, fig=fig)
fig.suptitle("")
plt.show()

# 6. Cross Validation

In [ ]:
dm = dmatrix(" + ".join(sms_vars) + ' + np.log(_price_doc)', df_train_macro_with_outliers, return_type="dataframe")
X = dm[dm.columns.drop(['np.log(_price_doc)'])]
y = dm['np.log(_price_doc)']
cv = cv = KFold(n_splits=1000, shuffle=True, random_state=0)
r2s = cross_val_score(SMWrapper(sm.OLS), X, y, scoring='r2', cv=cv)
r2s.mean()

In [ ]:
plt.hist(r2s, bins=100)

In [ ]:
y_pred = np.exp(result.predict(df_test_macro))
y_pred = y_pred.to_frame('price_doc')
y_pred.to_csv('./data/stats_models_{}.csv'.format(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')), header=True, index=True)

## Score

0.39773